In [27]:
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data_path = "./data/"



In [3]:
train_origin = pd.read_csv(data_path+"train.csv", sep="###__###",header = None,encoding="utf-8")
train_origin.columns = ['ID', 'Age', 'Gender', 'Education', 'Query_List']
train_origin.head()

E:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [4]:
train_origin.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          100000 non-null  object
 1   Age         100000 non-null  int64 
 2   Gender      100000 non-null  int64 
 3   Education   100000 non-null  int64 
 4   Query_List  100000 non-null  object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


In [5]:
train_origin["Age"].unique()



array([1, 2, 4, 3, 6, 5, 0], dtype=int64)

In [6]:
train_origin["Gender"].unique()



array([1, 2, 0], dtype=int64)

In [7]:
train_origin["Education"].unique()



array([4, 3, 0, 5, 2, 6, 1], dtype=int64)

In [8]:
train_origin["content"] = train_origin["Query_List"].apply(lambda  x:x.split("\t"))
train_origin.head()

,ID,Age,Gender,Education,Query_List,content
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,"[柔和双沟, 女生, 中财网首页 财经, http://pan.baidu.com/s/1p..."
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","[广州厨宝烤箱, 世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕, 厦门酒店用品批发市场, 我..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,"[钻石之泪耳机, 盘锦到沈阳, 旅顺公交, 辽宁阜新车牌, baidu, k715, k71..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,"[最受欢迎狗狗排行榜, 舶怎么读, 场景描 写范例, 三维绘图软件, 枣和酸奶能一起吃吗, ..."
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,"[干槽症能自愈吗, 太太万岁叶舒心去没去美国, 干槽症, 右眼皮下面一直跳是怎么回事, 麦当..."


In [20]:
def string_list_cut(strings):
    return_list = []
    for item in strings:
        if "pan.baidu.com" in item:
            return_list.append("百度网盘")
            continue
        if item == "," or item == "，":
            continue
        ele_list = jieba.lcut(item)
        if len(ele_list) < 1:
            continue
        return_list += ele_list
    return return_list

In [21]:
train_origin["cut_content"] = train_origin["content"].apply(lambda  x:string_list_cut(x))
train_origin.head()


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\10926\AppData\Local\Temp\jieba.cache
Loading model cost 0.669 seconds.
Prefix dict has been built successfully.


,ID,Age,Gender,Education,Query_List,content,cut_content
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,"[柔和双沟, 女生, 中财网首页 财经, http://pan.baidu.com/s/1p...","[柔和, 双沟, 女生, 中财网, 首页, , 财经, 百度网盘, 周公, 解梦, 大全,..."
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","[广州厨宝烤箱, 世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕, 厦门酒店用品批发市场, 我...","[广州, 厨宝, 烤箱, 世情, 薄, ,, 人情, 恶, ,, 雨送, 黄昏, 花易落, ..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...,"[钻石之泪耳机, 盘锦到沈阳, 旅顺公交, 辽宁阜新车牌, baidu, k715, k71...","[钻石, 之泪, 耳机, 盘锦, 到, 沈阳, 旅顺, 公交, 辽宁, 阜新, 车牌, ba..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,"[最受欢迎狗狗排行榜, 舶怎么读, 场景描 写范例, 三维绘图软件, 枣和酸奶能一起吃吗, ...","[最, 受欢迎, 狗狗, 排行榜, 舶, 怎么, 读, 场景, 描, , 写, 范例, 三..."
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,"[干槽症能自愈吗, 太太万岁叶舒心去没去美国, 干槽症, 右眼皮下面一直跳是怎么回事, 麦当...","[干槽症, 能, 自愈, 吗, 太太, 万岁, 叶, 舒心, 去, 没, 去, 美国, 干槽..."


In [ ]:
age_train_origin = train_origin[["Age", "cut_content"]]
age_train_origin.head()

In [ ]:
corpus = []
for index, row in age_train_origin.iterrows():
    corpus += row["cut_content"]

In [ ]:
tfidf_model = TfidfVectorizer
tfidf_model.fit(corpus)

In [ ]:
kf = KFold(n_splits=5)
for train, test in kf.split(age_train_origin):


,Age,cut_content
0,1,"[柔和, 双沟, 女生, 中财网, 首页, , 财经, 百度网盘, 周公, 解梦, 大全,..."
1,2,"[广州, 厨宝, 烤箱, 世情, 薄, ,, 人情, 恶, ,, 雨送, 黄昏, 花易落, ..."
2,4,"[钻石, 之泪, 耳机, 盘锦, 到, 沈阳, 旅顺, 公交, 辽宁, 阜新, 车牌, ba..."
3,4,"[最, 受欢迎, 狗狗, 排行榜, 舶, 怎么, 读, 场景, 描, , 写, 范例, 三..."
4,2,"[干槽症, 能, 自愈, 吗, 太太, 万岁, 叶, 舒心, 去, 没, 去, 美国, 干槽..."


In [35]:
corpus = []
for index, row in age_train_origin.iterrows():
    corpus += row["cut_content"]

In [36]:
tfidf_model = TfidfVectorizer
tfidf_model.fit(corpus)

TypeError: fit() missing 1 required positional argument: 'raw_documents'

In [26]:
kf = KFold(n_splits=5)
for train, test in kf.split(age_train_origin):


*****
80000
20000
*****
80000
20000
*****
80000
20000
*****
80000
20000
*****
80000
20000
